In [ ]:
!pip install -U sagemaker

In [ ]:
import boto3
import pandas as pd
import numpy as np
import sagemaker
import json
from sagemaker.sklearn.estimator import SKLearn

s3 = boto3.client("s3")
sagemaker_session = sagemaker.Session()

In [ ]:
training_job_name = 'sagemaker-scikit-learn-2024-06-04-06-44-38-978'

In [ ]:
model_name = f"model-{training_job_name}"
endpoint_name = f"endpoint-{training_job_name}"

In [ ]:
trained_estimator = SKLearn.attach(
    training_job_name=training_job_name,
    sagemaker_session=sagemaker_session,
)

In [ ]:
predictor = trained_estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    model_name=model_name,
    endpoint_name=endpoint_name,
)

In [ ]:
categories_map = {0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}

In [ ]:
client = sagemaker_session.sagemaker_runtime_client

request_body = {"Input": [[9.0, 3571, 1976, 0.525]]}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

response = client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="application/json", Body=payload
)

result = json.loads(response["Body"].read().decode())["Output"]

print("Predicted class category {} ({})".format(result, categories_map[result]))

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

# OR
predictor.delete_endpoint(delete_endpoint_config=False)